In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import re
import string
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from autocorrect import spell
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
#stemmer= PorterStemmer()
stemmer = SnowballStemmer("english")
lem = WordNetLemmatizer()
#https://www.kaggle.com/samdeeplearning/deepnlp#Sheet_1.csv

In [2]:
df = pd.read_csv("Sheet_1.csv")
df.head()

,response_id,class,response_text,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,response_1,not_flagged,I try and avoid this sort of conflict,NaN,NaN,NaN,NaN,NaN
1,response_2,flagged,Had a friend open up to me about his mental ad...,NaN,NaN,NaN,NaN,NaN
2,response_3,flagged,I saved a girl from suicide once. She was goin...,NaN,NaN,NaN,NaN,NaN
3,response_4,not_flagged,i cant think of one really...i think i may hav...,NaN,NaN,NaN,NaN,NaN
4,response_5,not_flagged,Only really one friend who doesn't fit into th...,,NaN,NaN,NaN,NaN


In [3]:
df.response_text.head(5)

0                I try and avoid this sort of conflict
1    Had a friend open up to me about his mental ad...
2    I saved a girl from suicide once. She was goin...
3    i cant think of one really...i think i may hav...
4    Only really one friend who doesn't fit into th...
Name: response_text, dtype: object

In [4]:
stop_words = set(stopwords.words('english'))

In [5]:
def clean_sentence(value):
    value = value.lower()
    value = re.sub(r'\d+', '', value)
    value = value.replace("n't", " not")
    value = value.replace("'s", "")
    value = re.sub(r'[^\w\s]','',value)
    value = value.strip()
    return value

In [6]:
df["response_text"] = df.response_text.apply(clean_sentence) 

In [7]:
def get_tokens(value):
    tokens = word_tokenize(value)
    return tokens

In [8]:
df["token"] = df.response_text.apply(get_tokens)

In [9]:
def remove_stop_words(value):
    filtered_words = [lem.lemmatize(w) for w in value if w not in stop_words] #lem.lemmatize(w)
    return filtered_words

In [10]:
df["token"] = df.token.apply(remove_stop_words)
df["str_token"] = df.token.apply(lambda x: " ".join(x))

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.8, stop_words=stopwords.words('english'))  
x2 = tfidfconverter.fit_transform(df["str_token"]).toarray()  
len(x2[0])

37

In [12]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [18]:
y = df["class"].values
df["class"].value_counts()

not_flagged    55
flagged        25
Name: class, dtype: int64

In [40]:
df["class"].values

array(['not_flagged', 'flagged', 'flagged', 'not_flagged', 'not_flagged',
       'not_flagged', 'flagged', 'flagged', 'not_flagged', 'flagged',
       'not_flagged', 'flagged', 'not_flagged', 'not_flagged',
       'not_flagged', 'flagged', 'not_flagged', 'not_flagged',
       'not_flagged', 'not_flagged', 'not_flagged', 'flagged', 'flagged',
       'not_flagged', 'not_flagged', 'flagged', 'flagged', 'not_flagged',
       'flagged', 'not_flagged', 'not_flagged', 'not_flagged',
       'not_flagged', 'not_flagged', 'not_flagged', 'not_flagged',
       'flagged', 'not_flagged', 'flagged', 'not_flagged', 'flagged',
       'not_flagged', 'not_flagged', 'not_flagged', 'not_flagged',
       'not_flagged', 'not_flagged', 'not_flagged', 'flagged',
       'not_flagged', 'not_flagged', 'flagged', 'not_flagged',
       'not_flagged', 'not_flagged', 'not_flagged', 'not_flagged',
       'flagged', 'not_flagged', 'flagged', 'not_flagged', 'flagged',
       'not_flagged', 'not_flagged', 'flagged', 'not

In [20]:
y = labelencoder.fit_transform(y)

In [39]:
y

array([1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1], dtype=int64)

In [21]:
mnb = MultinomialNB()

In [35]:
from sklearn.model_selection import cross_val_score
cross_val_score(mnb, x2, y, scoring='accuracy', cv=10).mean()# accuracy

0.7301587301587301

In [36]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(mnb, x2, y, cv=10)
conf_mat = confusion_matrix(y, y_pred)
conf_mat

array([[ 3, 22],
       [ 0, 55]], dtype=int64)

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y, y_pred))

             precision    recall  f1-score   support

          0       1.00      0.12      0.21        25
          1       0.71      1.00      0.83        55

avg / total       0.80      0.72      0.64        80

